In [121]:
import pandas as pd
import os
import psycopg2
import pandas as pd
import numpy as np
import psycopg2
import psycopg2.extras as extras
import os
import re


#path = 'C:/GitHub/Proyecto_Final_BI/'
path = ""

Lectura de archivos

In [122]:
# función dada en clase
def load_data(filename):
    """ Load dataset from filename
    Args:
        filename (string): Path to data
        
    Returns:
        df (DataFrame): Dataset from file
    """
    df = pd.read_csv(filename)
    return df

filename = os.path.join(path,'Benefits_Cost_Sharing_PUF.csv')
Benefits_CostSharing = load_data(filename)

filename = os.path.join(path,'Business_Rules_PUF.csv')
Business_Rules = load_data(filename)

filename = os.path.join(path,'Plan_Attributes_PUF.csv')
Plan_Attributes = load_data(filename)

filename = os.path.join(path,'Rate_PUF.csv')
Rate = load_data(filename)

/var/folders/nc/msnr18q57_bfryvsdz5vb_w00000gn/T/ipykernel_43017/278177419.py:10: DtypeWarning: Columns (9,12,19) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filename)
/var/folders/nc/msnr18q57_bfryvsdz5vb_w00000gn/T/ipykernel_43017/278177419.py:10: DtypeWarning: Columns (27,64,80,81,82,86,87,88,89,90,91,111,122,123,124,125,147,148) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filename)


# CMS tablas

In [123]:
Benefits_CostSharing = Benefits_CostSharing[['StateCode', 'IssuerId', 'StandardComponentId', 'PlanId', 'BenefitName', 
                                            'CopayInnTier1', 'CopayInnTier2', 'CopayOutofNet', 'CoinsInnTier1', 'CoinsInnTier2',
                                            'CoinsOutofNet', 'IsEHB', 'IsCovered', 'QuantLimitOnSvc', 'LimitQty','LimitUnit', 'Exclusions']]
Benefits_CostSharing.columns

Index(['StateCode', 'IssuerId', 'StandardComponentId', 'PlanId', 'BenefitName',
       'CopayInnTier1', 'CopayInnTier2', 'CopayOutofNet', 'CoinsInnTier1',
       'CoinsInnTier2', 'CoinsOutofNet', 'IsEHB', 'IsCovered',
       'QuantLimitOnSvc', 'LimitQty', 'LimitUnit', 'Exclusions'],
      dtype='object')

In [124]:
Business_Rules = Business_Rules[['StateCode', 'IssuerId', 'MedicalDentalBoth', 'StandardComponentId',
                                 'MaximumUnderageDependent', 'MarketCoverage',
                                 'DentalOnlyPlan']]
Business_Rules.columns

Index(['StateCode', 'IssuerId', 'MedicalDentalBoth', 'StandardComponentId',
       'MaximumUnderageDependent', 'MarketCoverage', 'DentalOnlyPlan'],
      dtype='object')

In [125]:
Plan_Attributes = Plan_Attributes[['StateCode', 'IssuerId', 'IssuerMarketPlaceMarketingName', 
                                   'MarketCoverage', 'StandardComponentId', 'PlanMarketingName', 'IsNewPlan', 'MetalLevel', 
                                   'ChildOnlyOffering', 'DiseaseManagementProgramsOffered',
                                   'EHBPercentTotalPremium', 'PlanId',
                                   'IssuerActuarialValue', 'IsHSAEligible']]
Plan_Attributes.columns

Index(['StateCode', 'IssuerId', 'IssuerMarketPlaceMarketingName',
       'MarketCoverage', 'StandardComponentId', 'PlanMarketingName',
       'IsNewPlan', 'MetalLevel', 'ChildOnlyOffering',
       'DiseaseManagementProgramsOffered', 'EHBPercentTotalPremium', 'PlanId',
       'IssuerActuarialValue', 'IsHSAEligible'],
      dtype='object')

In [126]:
Rate = Rate[['StateCode', 'IssuerId','PlanId','Age', 'IndividualRate', 'Couple',
       'PrimarySubscriberAndOneDependent', 'PrimarySubscriberAndTwoDependents',
       'PrimarySubscriberAndThreeOrMoreDependents', 'CoupleAndOneDependent',
       'CoupleAndTwoDependents', 'CoupleAndThreeOrMoreDependents' ]]
Rate.columns

Index(['StateCode', 'IssuerId', 'PlanId', 'Age', 'IndividualRate', 'Couple',
       'PrimarySubscriberAndOneDependent', 'PrimarySubscriberAndTwoDependents',
       'PrimarySubscriberAndThreeOrMoreDependents', 'CoupleAndOneDependent',
       'CoupleAndTwoDependents', 'CoupleAndThreeOrMoreDependents'],
      dtype='object')

La tabla de Business Rules PUF puede ser el primer filtro

In [127]:
Business_Rules.head(10)

,StateCode,IssuerId,MedicalDentalBoth,StandardComponentId,MaximumUnderageDependent,MarketCoverage,DentalOnlyPlan
0,AK,21989,Dental,21989AK0030001,3,Individual,Yes
1,AK,21989,Dental,21989AK0030051,3,SHOP (Small Group),Yes
2,AK,21989,Dental,21989AK0030052,3,SHOP (Small Group),Yes
3,AK,21989,Dental,21989AK0040051,3,SHOP (Small Group),Yes
4,AK,21989,Dental,21989AK0040052,3,SHOP (Small Group),Yes
5,AK,21989,Dental,21989AK0040053,3,SHOP (Small Group),Yes
6,AK,21989,Dental,21989AK0040054,3,SHOP (Small Group),Yes
7,AK,21989,Dental,21989AK0040055,3,SHOP (Small Group),Yes
8,AK,21989,Dental,21989AK0050001,3,Individual,Yes
9,AK,21989,Dental,21989AK0050002,3,Individual,Yes


In [128]:
Business_Rules.StateCode.unique()

array(['AK', 'AL', 'AZ', 'FL', 'GA', 'IN', 'LA', 'MO', 'MS', 'NC', 'OK',
       'TX', 'WI', 'IA', 'ND', 'SC', 'TN', 'WY', 'AR', 'DE', 'HI', 'IL',
       'NE', 'NH', 'OH', 'KS', 'MI', 'MT', 'OR', 'SD', 'UT', 'VA', 'WV'],
      dtype=object)

Virginia: 8642274 VA
Arizona: 7276316 AZ
Tennesse: 6975218 TN


In [129]:
Business_Rules[Business_Rules["MedicalDentalBoth"] == 'Both']

,StateCode,IssuerId,MedicalDentalBoth,StandardComponentId,MaximumUnderageDependent,MarketCoverage,DentalOnlyPlan
75,AL,46944,Both,46944AL0280001,3,SHOP (Small Group),No
76,AL,46944,Both,46944AL0340001,3,SHOP (Small Group),No
77,AL,46944,Both,46944AL0370001,3,Individual,No
78,AL,46944,Both,46944AL0380001,3,SHOP (Small Group),No
79,AL,46944,Both,46944AL0410001,3,Individual,No
...,...,...,...,...,...,...,...
7321,VA,41921,Both,41921VA0020071,3,Individual,No
7322,VA,41921,Both,41921VA0020072,3,Individual,No
7323,VA,41921,Both,41921VA0020073,3,Individual,No
7324,VA,41921,Both,41921VA0020074,3,Individual,No


In [130]:
Business_Rules = Business_Rules[(Business_Rules.StateCode.isin(['TX','FL','IL','OH','NC','GA','MI','VA','AZ','TN'])) & (Business_Rules.MedicalDentalBoth != 'Dental')]

In [131]:
Business_Rules.StateCode.unique()

array(['AZ', 'FL', 'GA', 'NC', 'TX', 'TN', 'IL', 'OH', 'MI', 'VA'],
      dtype=object)

In [132]:
Rate = Rate[Rate.PlanId.isin(Business_Rules.StandardComponentId)]

In [133]:
Rate.StateCode.unique()

array(['AZ', 'FL', 'GA', 'NC', 'TN', 'TX', 'IL', 'MI', 'OH', 'VA'],
      dtype=object)

In [134]:
Rate.head()

,StateCode,IssuerId,PlanId,Age,IndividualRate,Couple,PrimarySubscriberAndOneDependent,PrimarySubscriberAndTwoDependents,PrimarySubscriberAndThreeOrMoreDependents,CoupleAndOneDependent,CoupleAndTwoDependents,CoupleAndThreeOrMoreDependents
65110,AZ,13877,13877AZ0070002,0-14,200.78,NaN,NaN,NaN,NaN,NaN,NaN,NaN
65111,AZ,13877,13877AZ0070002,15,218.63,NaN,NaN,NaN,NaN,NaN,NaN,NaN
65112,AZ,13877,13877AZ0070002,16,225.45,NaN,NaN,NaN,NaN,NaN,NaN,NaN
65113,AZ,13877,13877AZ0070002,17,232.28,NaN,NaN,NaN,NaN,NaN,NaN,NaN
65114,AZ,13877,13877AZ0070002,18,239.63,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [135]:
Plan_Attributes = Plan_Attributes[Plan_Attributes.StandardComponentId.isin(Business_Rules.StandardComponentId)]

In [136]:
Plan_Attributes.StateCode.unique()

array(['AZ', 'FL', 'GA', 'IL', 'MI', 'NC', 'OH', 'TN', 'TX', 'VA'],
      dtype=object)

In [137]:
Plan_Attributes.ChildOnlyOffering.unique()

array(['Allows Adult and Child-Only', 'Allows Child-Only',
       'Allows Adult-Only'], dtype=object)

In [138]:
Benefits_CostSharing = Benefits_CostSharing[Benefits_CostSharing.StandardComponentId.isin(Business_Rules.StandardComponentId)]

In [139]:
Benefits_CostSharing[Benefits_CostSharing["PlanId"] == "19898FL0340008-01"]["CopayInnTier1"].unique()

array(['No Charge after deductible', nan, 'No Charge'], dtype=object)

In [140]:
state = {'StateCode': ['AZ', 'FL', 'GA', 'IL', 'MI', 'NC', 'OH', 'TN', 'TX', 'VA'], 'StateName': ["Arizona", "Florida", "Georgia", "Illinois", "Michigan", "NorthCarolina", "Ohio", "Tennessee", "Texas", "Virginia"]}
state = pd.DataFrame(data=state)
state

,StateCode,StateName
0,AZ,Arizona
1,FL,Florida
2,GA,Georgia
3,IL,Illinois
4,MI,Michigan
5,NC,NorthCarolina
6,OH,Ohio
7,TN,Tennessee
8,TX,Texas
9,VA,Virginia


Proceso de imputaciones

In [141]:
Benefits_CostSharing.IsEHB.unique()

array(['Yes', nan], dtype=object)

In [142]:
Benefits_CostSharing = Benefits_CostSharing.fillna("No")

In [143]:
Benefits_CostSharing.IsEHB.unique()

array(['Yes', 'No'], dtype=object)

In [144]:
for i in Plan_Attributes.columns:
    nas = Plan_Attributes[i].isna().sum()
    if nas>0:
        print("{0:<20}  {1}".format(i,nas))

DiseaseManagementProgramsOffered  2664
EHBPercentTotalPremium  276
IssuerActuarialValue  12498
IsHSAEligible         128


In [145]:
Plan_Attributes.fillna({'IsHSAEligible': "No"}, inplace=True)

In [146]:
for i in Plan_Attributes.columns:
    nas = Plan_Attributes[i].isna().sum()
    if nas>0:
        print("{0:<20}  {1}".format(i,nas))

DiseaseManagementProgramsOffered  2664
EHBPercentTotalPremium  276
IssuerActuarialValue  12498


In [147]:
Plan_Attributes['IssuerActuarialValue'] = Plan_Attributes.iloc[:,12:-1].apply(lambda x: x.str.replace('%', '')).astype(float)
Plan_Attributes.dtypes

StateCode                            object
IssuerId                              int64
IssuerMarketPlaceMarketingName       object
MarketCoverage                       object
StandardComponentId                  object
PlanMarketingName                    object
IsNewPlan                            object
MetalLevel                           object
ChildOnlyOffering                    object
DiseaseManagementProgramsOffered     object
EHBPercentTotalPremium              float64
PlanId                               object
IssuerActuarialValue                float64
IsHSAEligible                        object
dtype: object

In [148]:
Plan_Attributes.loc[Plan_Attributes['MetalLevel'] == 'Catastrophic', 'IssuerActuarialValue'] = Plan_Attributes.loc[Plan_Attributes['MetalLevel'] == 'Catastrophic', 'IssuerActuarialValue'].fillna(Plan_Attributes[Plan_Attributes['MetalLevel'] == 'Catastrophic']['IssuerActuarialValue'].mean())
Plan_Attributes.loc[Plan_Attributes['MetalLevel'] == 'Bronze', 'IssuerActuarialValue'] = Plan_Attributes.loc[Plan_Attributes['MetalLevel'] == 'Bronze', 'IssuerActuarialValue'].fillna(Plan_Attributes[Plan_Attributes['MetalLevel'] == 'Bronze']['IssuerActuarialValue'].mean())
Plan_Attributes.loc[Plan_Attributes['MetalLevel'] == 'Expanded Bronze', 'IssuerActuarialValue'] = Plan_Attributes.loc[Plan_Attributes['MetalLevel'] == 'Expanded Bronze', 'IssuerActuarialValue'].fillna(Plan_Attributes[Plan_Attributes['MetalLevel'] == 'Expanded Bronze']['IssuerActuarialValue'].mean())
Plan_Attributes.loc[Plan_Attributes['MetalLevel'] == 'Gold', 'IssuerActuarialValue'] = Plan_Attributes.loc[Plan_Attributes['MetalLevel'] == 'Gold', 'IssuerActuarialValue'].fillna(Plan_Attributes[Plan_Attributes['MetalLevel'] == 'Gold']['IssuerActuarialValue'].mean())
Plan_Attributes.loc[Plan_Attributes['MetalLevel'] == 'Silver', 'IssuerActuarialValue'] = Plan_Attributes.loc[Plan_Attributes['MetalLevel'] == 'Silver', 'IssuerActuarialValue'].fillna(Plan_Attributes[Plan_Attributes['MetalLevel'] == 'Silver']['IssuerActuarialValue'].mean())
Plan_Attributes.loc[Plan_Attributes['MetalLevel'] == 'Platinum', 'IssuerActuarialValue'] = Plan_Attributes.loc[Plan_Attributes['MetalLevel'] == 'Platinum', 'IssuerActuarialValue'].fillna(Plan_Attributes[Plan_Attributes['MetalLevel'] == 'Platinum']['IssuerActuarialValue'].mean())
Plan_Attributes.loc[Plan_Attributes['MetalLevel'] == 'Low', 'IssuerActuarialValue'] = Plan_Attributes.loc[Plan_Attributes['MetalLevel'] == 'Low', 'IssuerActuarialValue'].fillna(Plan_Attributes[Plan_Attributes['MetalLevel'] == 'Low']['IssuerActuarialValue'].mean())
Plan_Attributes.loc[Plan_Attributes['MetalLevel'] == 'High', 'IssuerActuarialValue'] = Plan_Attributes.loc[Plan_Attributes['MetalLevel'] == 'High', 'IssuerActuarialValue'].fillna(Plan_Attributes[Plan_Attributes['MetalLevel'] == 'High']['IssuerActuarialValue'].mean())

In [149]:
Plan_Attributes[Plan_Attributes["IssuerActuarialValue"].isna()]["MetalLevel"].unique()

array(['Catastrophic', 'High'], dtype=object)

In [150]:
for i in Plan_Attributes.columns:
    nas = Plan_Attributes[i].isna().sum()
    if nas>0:
        print("{0:<20}  {1}".format(i,nas))

DiseaseManagementProgramsOffered  2664
EHBPercentTotalPremium  276
IssuerActuarialValue  307


In [151]:
Plan_Attributes.fillna({'IssuerActuarialValue': 0.0}, inplace=True)

In [152]:
for i in Plan_Attributes.columns:
    nas = Plan_Attributes[i].isna().sum()
    if nas>0:
        print("{0:<20}  {1}".format(i,nas))

DiseaseManagementProgramsOffered  2664
EHBPercentTotalPremium  276


In [153]:
Plan_Attributes.fillna({'EHBPercentTotalPremium': Plan_Attributes["EHBPercentTotalPremium"].mean()}, inplace=True)

In [154]:
for i in Plan_Attributes.columns:
    nas = Plan_Attributes[i].isna().sum()
    if nas>0:
        print("{0:<20}  {1}".format(i,nas))

DiseaseManagementProgramsOffered  2664


In [155]:
Plan_Attributes.fillna({'DiseaseManagementProgramsOffered': ""}, inplace=True)

In [156]:
for i in Plan_Attributes.columns:
    nas = Plan_Attributes[i].isna().sum()
    if nas>0:
        print("{0:<20}  {1}".format(i,nas))

In [157]:
for i in Rate.columns:
    nas = Rate[i].isna().sum()
    if nas>0:
        print("{0:<20}  {1}".format(i,nas))

Couple                1209465
PrimarySubscriberAndOneDependent  1209465
PrimarySubscriberAndTwoDependents  1209465
PrimarySubscriberAndThreeOrMoreDependents  1209465
CoupleAndOneDependent  1209465
CoupleAndTwoDependents  1209465
CoupleAndThreeOrMoreDependents  1209465


In [158]:
Rate.Age.unique()

array(['0-14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24',
       '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35',
       '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46',
       '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57',
       '58', '59', '60', '61', '62', '63', '64 and over'], dtype=object)

In [159]:
Rate.loc[Rate['Age'] == '0-14', 'Age'] = '14'
Rate.loc[Rate['Age'] == '64 and over', 'Age'] = '64'

In [160]:
Rate['Age'] = Rate['Age'].astype(int)
Rate.Age.unique()


array([14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30,
       31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47,
       48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64])

* análisis corto

In [161]:
Plan_Attributes.MetalLevel.unique()

array(['Expanded Bronze', 'Silver', 'Catastrophic', 'Gold', 'Bronze',
       'Platinum', 'Low', 'High'], dtype=object)

In [162]:
# orden peor a mejor
# Catastrophic: 0.0
# High: 0.0
# Low: 70.152
# Bronze: 70.6125
# Expanded Bronze: 73.154
# Silver: 80.967
# Gold: 84.588
# Platinum: 92.965
#Plan_Attributes[(Plan_Attributes['MetalLevel'] == "Platinum")]["IssuerActuarialValue"].mean()#& (~Plan_Attributes['IssuerActuarialValue'].isna())]

In [163]:
Business_Rules.StandardComponentId.nunique()

3941

In [164]:
Plan_Attributes.PlanId.nunique()

19188

In [165]:
# SELECE BR.StateCode, BR.StandardComponentId, PA.PlanId, PA.MetalLevel, 
    # PA.PlanMarketingName, PA.IsNewPlan, BA.MaximumUnderageDependent, BA.MarketCoverge, 
    # PA.ChildOnlyOffering, PA.DiseaseManagementProgramsOffered, PA.EHBPercentTotalPremium, PA.IsHSAEligible
# FROM Business_Rule BR
# JOIN Plan_Attributes PA
# ON BR.StandardComponentId = PA.StandardComponentId

In [166]:
Planes = Business_Rules.merge(Plan_Attributes,left_on='StandardComponentId', right_on='StandardComponentId', how='inner')[["StateCode_x", "StandardComponentId", "PlanId", "MetalLevel",  "PlanMarketingName", "IsNewPlan", "MaximumUnderageDependent", "MarketCoverage_y", "ChildOnlyOffering", "DiseaseManagementProgramsOffered", "EHBPercentTotalPremium", "IsHSAEligible"]]

In [167]:
Planes

,StateCode_x,StandardComponentId,PlanId,MetalLevel,PlanMarketingName,IsNewPlan,MaximumUnderageDependent,MarketCoverage_y,ChildOnlyOffering,DiseaseManagementProgramsOffered,EHBPercentTotalPremium,IsHSAEligible
0,AZ,13877AZ0070002,13877AZ0070002-01,Expanded Bronze,Bronze Classic- PCP Saver,Existing,3,Individual,Allows Adult and Child-Only,"Asthma, Heart Disease, Depression, Diabetes, H...",1.0000,No
1,AZ,13877AZ0070002,13877AZ0070002-02,Expanded Bronze,Bronze Classic- PCP Saver,Existing,3,Individual,Allows Adult and Child-Only,"Asthma, Heart Disease, Depression, Diabetes, H...",1.0000,No
2,AZ,13877AZ0070002,13877AZ0070002-00,Expanded Bronze,Bronze Classic- PCP Saver,Existing,3,Individual,Allows Adult and Child-Only,"Asthma, Heart Disease, Depression, Diabetes, H...",1.0000,No
3,AZ,13877AZ0070002,13877AZ0070002-03,Expanded Bronze,Bronze Classic- PCP Saver,Existing,3,Individual,Allows Adult and Child-Only,"Asthma, Heart Disease, Depression, Diabetes, H...",1.0000,No
4,AZ,13877AZ0070003,13877AZ0070003-01,Expanded Bronze,Bronze Classic,Existing,3,Individual,Allows Adult and Child-Only,"Asthma, Heart Disease, Depression, Diabetes, H...",1.0000,No
...,...,...,...,...,...,...,...,...,...,...,...,...
19183,VA,95185VA0530023,95185VA0530023-06,Silver,KP VA Standard Silver 5800/40/Vision,New,3,Individual,Allows Adult and Child-Only,"Asthma, Heart Disease, Depression, Diabetes, H...",0.9887,No
19184,VA,95185VA0530024,95185VA0530024-03,Expanded Bronze,KP VA Standard Expanded Bronze 7500/50/Vision,New,3,Individual,Allows Adult and Child-Only,"Asthma, Heart Disease, Depression, Diabetes, H...",0.9887,No
19185,VA,95185VA0530024,95185VA0530024-02,Expanded Bronze,KP VA Standard Expanded Bronze 7500/50/Vision,New,3,Individual,Allows Adult and Child-Only,"Asthma, Heart Disease, Depression, Diabetes, H...",0.9887,No
19186,VA,95185VA0530024,95185VA0530024-00,Expanded Bronze,KP VA Standard Expanded Bronze 7500/50/Vision,New,3,Individual,Allows Adult and Child-Only,"Asthma, Heart Disease, Depression, Diabetes, H...",0.9887,No


In [168]:
Planes = Planes.rename(columns={'StateCode_x': 'StateCode', 'MarketCoverage_y': 'MarketCoverage'})
Planes.columns

Index(['StateCode', 'StandardComponentId', 'PlanId', 'MetalLevel',
       'PlanMarketingName', 'IsNewPlan', 'MaximumUnderageDependent',
       'MarketCoverage', 'ChildOnlyOffering',
       'DiseaseManagementProgramsOffered', 'EHBPercentTotalPremium',
       'IsHSAEligible'],
      dtype='object')

In [169]:
Planes.shape[0]

19188

In [170]:
vector_contados = []
for i in range(Planes.shape[0]):
    valor = Planes.iat[i,9]
    count_commas = valor.count(",")
    if (count_commas == 0):
        if(len(valor) > 0):
            vector_contados.append(count_commas+1)
        else:
            vector_contados.append(count_commas)
    else:
        vector_contados.append(count_commas+1)

columan_nueva = pd.DataFrame({'DiseaseManagementProgramsOffered': vector_contados})
#columan_nueva

# replace column 'B' with column 'C'
Planes['DiseaseManagementProgramsOffered'] = columan_nueva['DiseaseManagementProgramsOffered']
Planes

,StateCode,StandardComponentId,PlanId,MetalLevel,PlanMarketingName,IsNewPlan,MaximumUnderageDependent,MarketCoverage,ChildOnlyOffering,DiseaseManagementProgramsOffered,EHBPercentTotalPremium,IsHSAEligible
0,AZ,13877AZ0070002,13877AZ0070002-01,Expanded Bronze,Bronze Classic- PCP Saver,Existing,3,Individual,Allows Adult and Child-Only,9,1.0000,No
1,AZ,13877AZ0070002,13877AZ0070002-02,Expanded Bronze,Bronze Classic- PCP Saver,Existing,3,Individual,Allows Adult and Child-Only,9,1.0000,No
2,AZ,13877AZ0070002,13877AZ0070002-00,Expanded Bronze,Bronze Classic- PCP Saver,Existing,3,Individual,Allows Adult and Child-Only,9,1.0000,No
3,AZ,13877AZ0070002,13877AZ0070002-03,Expanded Bronze,Bronze Classic- PCP Saver,Existing,3,Individual,Allows Adult and Child-Only,9,1.0000,No
4,AZ,13877AZ0070003,13877AZ0070003-01,Expanded Bronze,Bronze Classic,Existing,3,Individual,Allows Adult and Child-Only,9,1.0000,No
...,...,...,...,...,...,...,...,...,...,...,...,...
19183,VA,95185VA0530023,95185VA0530023-06,Silver,KP VA Standard Silver 5800/40/Vision,New,3,Individual,Allows Adult and Child-Only,9,0.9887,No
19184,VA,95185VA0530024,95185VA0530024-03,Expanded Bronze,KP VA Standard Expanded Bronze 7500/50/Vision,New,3,Individual,Allows Adult and Child-Only,9,0.9887,No
19185,VA,95185VA0530024,95185VA0530024-02,Expanded Bronze,KP VA Standard Expanded Bronze 7500/50/Vision,New,3,Individual,Allows Adult and Child-Only,9,0.9887,No
19186,VA,95185VA0530024,95185VA0530024-00,Expanded Bronze,KP VA Standard Expanded Bronze 7500/50/Vision,New,3,Individual,Allows Adult and Child-Only,9,0.9887,No


In [171]:
Planes['DiseaseManagementProgramsOffered'] = Planes['DiseaseManagementProgramsOffered'].astype(object)

In [165]:
Rate = Rate.rename(columns={'PlanId': 'StandardComponentId_PlanId'})
Rate = Rate.reset_index()
Rate = Rate.drop(columns=['index'])
Rate.columns


Index(['StateCode', 'IssuerId', 'StandardComponentId_PlanId', 'Age',
       'IndividualRate', 'Couple', 'PrimarySubscriberAndOneDependent',
       'PrimarySubscriberAndTwoDependents',
       'PrimarySubscriberAndThreeOrMoreDependents', 'CoupleAndOneDependent',
       'CoupleAndTwoDependents', 'CoupleAndThreeOrMoreDependents'],
      dtype='object')

In [166]:
Planes = Planes.assign(idplan = lambda df: df.index+1)
Planes = Planes[ ['idplan', 'StateCode', 'StandardComponentId', 'PlanId', 'MetalLevel',
       'PlanMarketingName', 'IsNewPlan', 'MaximumUnderageDependent',
       'MarketCoverage', 'ChildOnlyOffering',
       'DiseaseManagementProgramsOffered', 'EHBPercentTotalPremium',
       'IsHSAEligible']]
Rate = Rate.assign(idRate  = lambda df: df.index+1)
Rate = Rate[['idRate','StateCode', 'IssuerId', 'StandardComponentId_PlanId', 'Age',
       'IndividualRate', 'Couple', 'PrimarySubscriberAndOneDependent',
       'PrimarySubscriberAndTwoDependents',
       'PrimarySubscriberAndThreeOrMoreDependents', 'CoupleAndOneDependent',
       'CoupleAndTwoDependents', 'CoupleAndThreeOrMoreDependents']]
Benefits_CostSharing = Benefits_CostSharing.reset_index()
Benefits_CostSharing = Benefits_CostSharing.drop(columns=['index'])     
Benefits_CostSharing = Benefits_CostSharing.assign(idBeneficio = lambda df: df.index+1)
Benefits_CostSharing = Benefits_CostSharing[['idBeneficio','StateCode', 'IssuerId', 'StandardComponentId', 'PlanId', 'BenefitName',
       'CopayInnTier1', 'CopayInnTier2', 'CopayOutofNet', 'CoinsInnTier1',
       'CoinsInnTier2', 'CoinsOutofNet', 'IsEHB', 'IsCovered',
       'QuantLimitOnSvc', 'LimitQty', 'LimitUnit', 'Exclusions']]

In [167]:
#select airport_ident, type, description, frequency_mhz from airport_freq join airports on airport_freq.airport_ref = airports.id where airports.ident = 'KLAX'
#airport_freq.merge(airports[airports.ident == 'KLAX'][['id']], left_on='airport_ref', right_on='id', how='inner')[['airport_ident', 'type', 'description', 'frequency_mhz']]

Planes_beneficios = Planes.merge(Benefits_CostSharing[Benefits_CostSharing["IsCovered"] == 'Covered'], left_on="PlanId", right_on="PlanId", how='inner')[['idBeneficio','idplan', 'BenefitName', 'IsEHB', 'IsCovered']]
Planes_beneficios

,idBeneficio,idplan,BenefitName,IsEHB,IsCovered
0,241,1,Primary Care Visit to Treat an Injury or Illness,Yes,Covered
1,242,1,Specialist Visit,Yes,Covered
2,243,1,"Other Practitioner Office Visit (Nurse, Physic...",Yes,Covered
3,244,1,"Outpatient Facility Fee (e.g., Ambulatory Surg...",Yes,Covered
4,245,1,Outpatient Surgery Physician/Surgical Services,Yes,Covered
...,...,...,...,...,...
1055542,1358439,19188,Prosthetic Devices,Yes,Covered
1055543,1358440,19188,Infusion Therapy,Yes,Covered
1055544,1358441,19188,Treatment for Temporomandibular Joint Disorders,Yes,Covered
1055545,1358442,19188,Nutritional Counseling,Yes,Covered


# Census

In [168]:
filename = os.path.join(path,'Age.csv')
age = load_data(filename)

In [169]:
age.head()

,Label (Grouping),Arizona!!Total!!Estimate,Arizona!!Percent!!Estimate,Arizona!!Male!!Estimate,Arizona!!Percent Male!!Estimate,Arizona!!Female!!Estimate,Arizona!!Percent Female!!Estimate,Florida!!Total!!Estimate,Florida!!Percent!!Estimate,Florida!!Male!!Estimate,...,Texas!!Male!!Estimate,Texas!!Percent Male!!Estimate,Texas!!Female!!Estimate,Texas!!Percent Female!!Estimate,Virginia!!Total!!Estimate,Virginia!!Percent!!Estimate,Virginia!!Male!!Estimate,Virginia!!Percent Male!!Estimate,Virginia!!Female!!Estimate,Virginia!!Percent Female!!Estimate
0,Total population,"7,276,316",(X),"3,629,620",(X),"3,646,696",(X),"21,781,128",(X),"10,714,520",...,"14,739,011",(X),"14,788,930",(X),"8,642,274",(X),"4,273,931",(X),"4,368,343",(X)
1,AGE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Under 5 years,"402,255",5.5%,"203,199",5.6%,"199,056",5.5%,"1,091,954",5.0%,"558,584",...,"967,416",6.6%,"924,211",6.2%,"480,992",5.6%,"245,127",5.7%,"235,865",5.4%
3,5 to 9 years,"446,309",6.1%,"232,179",6.4%,"214,130",5.9%,"1,146,199",5.3%,"584,591",...,"1,059,471",7.2%,"996,336",6.7%,"514,219",6.0%,"262,391",6.1%,"251,828",5.8%
4,10 to 14 years,"476,245",6.5%,"238,758",6.6%,"237,487",6.5%,"1,291,404",5.9%,"663,644",...,"1,119,382",7.6%,"1,087,144",7.4%,"559,020",6.5%,"292,819",6.9%,"266,201",6.1%


In [170]:
cont = 0
pasos = 2
age_data = age.iloc[2:20,[0]]
for cont in range(10):
    age_data.insert(cont+1, age.columns[pasos], age.iloc[2:20,[pasos]], True)
    pasos += 6
    cont += 1
age_data.head()


,Label (Grouping),Arizona!!Percent!!Estimate,Florida!!Percent!!Estimate,Georgia!!Percent!!Estimate,Illinois!!Percent!!Estimate,Michigan!!Percent!!Estimate,North Carolina!!Percent!!Estimate,Ohio!!Percent!!Estimate,Tennessee!!Percent!!Estimate,Texas!!Percent!!Estimate,Virginia!!Percent!!Estimate
2,Under 5 years,5.5%,5.0%,5.8%,5.6%,5.4%,5.5%,5.7%,5.7%,6.4%,5.6%
3,5 to 9 years,6.1%,5.3%,6.2%,6.2%,5.8%,5.9%,6.1%,6.1%,7.0%,6.0%
4,10 to 14 years,6.5%,5.9%,7.2%,6.4%,6.3%,6.5%,6.4%,6.4%,7.5%,6.5%
5,15 to 19 years,6.6%,5.7%,7.1%,6.7%,6.5%,6.7%,6.5%,6.4%,7.2%,6.6%
6,20 to 24 years,6.7%,5.7%,6.7%,6.3%,6.6%,6.7%,6.3%,6.4%,6.8%,6.6%


In [171]:
def vector_states(data):
    States = []
    for i in list(data.columns.values):
        try:
            limit = i.index('!')
            state = i[0:limit]
            if state in States:
                pass
            else:
                States.append(state)
        except:
            pass
    
    return States

In [172]:
States = vector_states(age_data)
age_data.rename(columns = {'Label (Grouping)':'ageRange'}, inplace = True)
for i in list(age_data.columns.values):
    for j in States:
        if i.find(j) != -1:
            age_data.rename(columns={i:j},inplace=True)

age_data

,ageRange,Arizona,Florida,Georgia,Illinois,Michigan,North Carolina,Ohio,Tennessee,Texas,Virginia
2,Under 5 years,5.5%,5.0%,5.8%,5.6%,5.4%,5.5%,5.7%,5.7%,6.4%,5.6%
3,5 to 9 years,6.1%,5.3%,6.2%,6.2%,5.8%,5.9%,6.1%,6.1%,7.0%,6.0%
4,10 to 14 years,6.5%,5.9%,7.2%,6.4%,6.3%,6.5%,6.4%,6.4%,7.5%,6.5%
5,15 to 19 years,6.6%,5.7%,7.1%,6.7%,6.5%,6.7%,6.5%,6.4%,7.2%,6.6%
6,20 to 24 years,6.7%,5.7%,6.7%,6.3%,6.6%,6.7%,6.3%,6.4%,6.8%,6.6%
7,25 to 29 years,6.9%,6.0%,6.7%,6.7%,6.5%,6.4%,6.5%,6.8%,7.1%,6.6%
8,30 to 34 years,6.8%,6.5%,6.8%,6.8%,6.5%,6.7%,6.6%,6.7%,7.3%,6.8%
9,35 to 39 years,6.5%,6.2%,6.8%,6.8%,6.0%,6.4%,6.3%,6.7%,7.2%,6.9%
10,40 to 44 years,6.1%,6.3%,6.6%,6.5%,6.0%,6.5%,6.1%,6.3%,6.8%,6.7%
11,45 to 49 years,5.8%,6.0%,6.4%,6.1%,5.7%,6.2%,5.8%,6.1%,6.2%,6.1%


In [173]:
age_data = age_data.melt('ageRange', var_name='Estados', value_name='porcentaje')
age_data

,ageRange,Estados,porcentaje
0,Under 5 years,Arizona,5.5%
1,5 to 9 years,Arizona,6.1%
2,10 to 14 years,Arizona,6.5%
3,15 to 19 years,Arizona,6.6%
4,20 to 24 years,Arizona,6.7%
...,...,...,...
175,65 to 69 years,Virginia,5.4%
176,70 to 74 years,Virginia,4.5%
177,75 to 79 years,Virginia,2.9%
178,80 to 84 years,Virginia,1.7%


In [174]:
dfAges = pd.DataFrame()
age_lower  = []
age_uper = []
bandera = True
for item in age_data.ageRange:
    numbers = re.findall(r'\d+', item)
    if (int(numbers[0]) == 5 and bandera):
        age_lower.append(-1)
        age_uper.append(int(numbers[0]))
        bandera = False
    else:
        age_lower.append(int(numbers[0]))
        if len(numbers) > 1:
            age_uper.append(int(numbers[1]))
        else:
            age_uper.append(-1)
dfAges['age_lower'] = age_lower
dfAges['age_uper'] = age_uper
dfAges

,age_lower,age_uper
0,-1,5
1,5,9
2,10,14
3,15,19
4,20,24
...,...,...
175,65,69
176,70,74
177,75,79
178,80,84


In [175]:
age_data = age_data.drop(columns = ['ageRange'])
age_data = age_data.reset_index()
age_data = age_data.drop(columns = ['index'])
age_data.insert(0, "age_lower", dfAges['age_lower'], True)
age_data.insert(1, "age_upper", dfAges['age_uper'], True)
age_data

,age_lower,age_upper,Estados,porcentaje
0,-1,5,Arizona,5.5%
1,5,9,Arizona,6.1%
2,10,14,Arizona,6.5%
3,15,19,Arizona,6.6%
4,20,24,Arizona,6.7%
...,...,...,...,...
175,65,69,Virginia,5.4%
176,70,74,Virginia,4.5%
177,75,79,Virginia,2.9%
178,80,84,Virginia,1.7%


In [176]:
age_data = age_data[['Estados', 'age_lower', 'age_upper', 'porcentaje']]
age_data

,Estados,age_lower,age_upper,porcentaje
0,Arizona,-1,5,5.5%
1,Arizona,5,9,6.1%
2,Arizona,10,14,6.5%
3,Arizona,15,19,6.6%
4,Arizona,20,24,6.7%
...,...,...,...,...
175,Virginia,65,69,5.4%
176,Virginia,70,74,4.5%
177,Virginia,75,79,2.9%
178,Virginia,80,84,1.7%


In [177]:
iniciales = ['AZ','FL','GA','IL','MI','NC','OH','TN','TX','VA']
for i in range(len(States)):
    age_data['Estados'] = age_data['Estados'].str.replace(States[i], iniciales[i])

age_data

,Estados,age_lower,age_upper,porcentaje
0,AZ,-1,5,5.5%
1,AZ,5,9,6.1%
2,AZ,10,14,6.5%
3,AZ,15,19,6.6%
4,AZ,20,24,6.7%
...,...,...,...,...
175,VA,65,69,5.4%
176,VA,70,74,4.5%
177,VA,75,79,2.9%
178,VA,80,84,1.7%


In [178]:
age_data.porcentaje = age_data.iloc[:,[3]].apply(lambda x: x.str.replace('%', '')).astype(float)/100

In [179]:
#select * from age where (age_lower > 24 and age_lower < 41);

In [180]:
filename = os.path.join(path,'Income.csv')
income = load_data(filename)

In [181]:
States = vector_states(income)
income.rename(columns = {'Label (Grouping)':'incomeRange'}, inplace = True)
for i in list(income.columns.values):
    for j in States:
        if i.find(j) != -1:
            income.rename(columns={i:j},inplace=True)

income

,incomeRange,Arizona,Florida,Georgia,Illinois,Michigan,North Carolina,Ohio,Tennessee,Texas,Virginia
0,Total:,"1,731,825","4,993,684","2,180,439","3,254,690","3,098,589","2,738,642","3,823,829","2,082,964","4,948,038","2,140,021"
1,Householder under 25 years:,"58,847","109,780","81,054","102,200","111,880","106,438","138,824","83,822","205,342","69,344"
2,"Less than $10,000","6,346","16,159","9,352","16,080","13,407","17,991","15,007","12,033","32,905","9,477"
3,"$10,000 to $14,999","3,510","5,749","4,534","6,988","7,588","4,746","5,285","4,558","10,771","4,493"
4,"$15,000 to $19,999","3,103","5,820","2,821","7,449","7,129","6,993","8,750","4,435","11,228","3,011"
...,...,...,...,...,...,...,...,...,...,...,...
64,"$75,000 to $99,999","80,814","238,733","84,399","115,957","118,424","96,981","132,395","67,824","185,931","78,490"
65,"$100,000 to $124,999","54,378","158,038","45,274","79,912","63,062","57,921","74,114","38,711","109,551","56,935"
66,"$125,000 to $149,999","34,035","98,829","30,017","51,230","40,146","36,750","42,616","22,635","78,452","36,904"
67,"$150,000 to $199,999","29,351","98,273","34,708","52,465","34,822","36,914","39,415","22,241","79,774","43,209"


In [182]:
income.iloc[:,1:].apply(lambda x: x.str.replace(',', '')).astype(int)

,Arizona,Florida,Georgia,Illinois,Michigan,North Carolina,Ohio,Tennessee,Texas,Virginia
0,1731825,4993684,2180439,3254690,3098589,2738642,3823829,2082964,4948038,2140021
1,58847,109780,81054,102200,111880,106438,138824,83822,205342,69344
2,6346,16159,9352,16080,13407,17991,15007,12033,32905,9477
3,3510,5749,4534,6988,7588,4746,5285,4558,10771,4493
4,3103,5820,2821,7449,7129,6993,8750,4435,11228,3011
...,...,...,...,...,...,...,...,...,...,...
64,80814,238733,84399,115957,118424,96981,132395,67824,185931,78490
65,54378,158038,45274,79912,63062,57921,74114,38711,109551,56935
66,34035,98829,30017,51230,40146,36750,42616,22635,78452,36904
67,29351,98273,34708,52465,34822,36914,39415,22241,79774,43209


In [183]:
income = income.melt('incomeRange', var_name='Estados', value_name='cantidad')
income

,incomeRange,Estados,cantidad
0,Total:,Arizona,"1,731,825"
1,Householder under 25 years:,Arizona,"58,847"
2,"Less than $10,000",Arizona,"6,346"
3,"$10,000 to $14,999",Arizona,"3,510"
4,"$15,000 to $19,999",Arizona,"3,103"
...,...,...,...
685,"$75,000 to $99,999",Virginia,"78,490"
686,"$100,000 to $124,999",Virginia,"56,935"
687,"$125,000 to $149,999",Virginia,"36,904"
688,"$150,000 to $199,999",Virginia,"43,209"


In [184]:
df_income = pd.DataFrame()
baseLimit  = []
upperLimit = []
bandera = True
bandera2 = True
for item in income.incomeRange:
    numbers = re.findall(r"\d{1,3}(?:,\d{3})*(?:\.\d{2})?", item)
    #print(numbers)
    if len(numbers)>0:
        if (int(numbers[0].replace(",", "")) == 25 and bandera):
            baseLimit.append(0)
            upperLimit.append(int(numbers[0].replace(",", "")))
            bandera = False
        elif (int(numbers[0].replace(",", "")) == 10000 and bandera2):
            baseLimit.append(0)
            upperLimit.append(int(numbers[0].replace(",", "")))
            bandera2 = False
        else:
            baseLimit.append(int(numbers[0].replace(",", "")))
            if len(numbers) > 1:
                upperLimit.append(int(numbers[1].replace(",", "")))
            else:
                upperLimit.append(-1)
                bandera2 = True
    else:
        baseLimit.append(-2)
        upperLimit.append(-2)
        bandera = True
        bandera2 = True
df_income['baseLimit'] = baseLimit
df_income['upperLimit'] = upperLimit
df_income

,baseLimit,upperLimit
0,-2,-2
1,0,25
2,0,10000
3,10000,14999
4,15000,19999
...,...,...
685,75000,99999
686,100000,124999
687,125000,149999
688,150000,199999


In [185]:
income = income.drop(columns = ['incomeRange'])
income = income.reset_index()
income = income.drop(columns = ['index'])
income.insert(0, "income_lower", df_income['baseLimit'], True)
income.insert(1, "income_upper", df_income['upperLimit'], True)
income

,income_lower,income_upper,Estados,cantidad
0,-2,-2,Arizona,"1,731,825"
1,0,25,Arizona,"58,847"
2,0,10000,Arizona,"6,346"
3,10000,14999,Arizona,"3,510"
4,15000,19999,Arizona,"3,103"
...,...,...,...,...
685,75000,99999,Virginia,"78,490"
686,100000,124999,Virginia,"56,935"
687,125000,149999,Virginia,"36,904"
688,150000,199999,Virginia,"43,209"


In [186]:
iniciales = ['AZ','FL','GA','IL','MI','NC','OH','TN','TX','VA']
for i in range(len(States)):
    income['Estados'] = income['Estados'].str.replace(States[i], iniciales[i])

income

,income_lower,income_upper,Estados,cantidad
0,-2,-2,AZ,"1,731,825"
1,0,25,AZ,"58,847"
2,0,10000,AZ,"6,346"
3,10000,14999,AZ,"3,510"
4,15000,19999,AZ,"3,103"
...,...,...,...,...
685,75000,99999,VA,"78,490"
686,100000,124999,VA,"56,935"
687,125000,149999,VA,"36,904"
688,150000,199999,VA,"43,209"


In [187]:
income['cantidad'] = income.iloc[:,3:].apply(lambda x: x.str.replace(',', '')).astype(int)
income

,income_lower,income_upper,Estados,cantidad
0,-2,-2,AZ,1731825
1,0,25,AZ,58847
2,0,10000,AZ,6346
3,10000,14999,AZ,3510
4,15000,19999,AZ,3103
...,...,...,...,...
685,75000,99999,VA,78490
686,100000,124999,VA,56935
687,125000,149999,VA,36904
688,150000,199999,VA,43209


### Prueba de obtener el rango de income más probable por estado

In [188]:
income[income['Estados'] == 'TX'].head(10)

,income_lower,income_upper,Estados,cantidad
552,-2,-2,TX,4948038
553,0,25,TX,205342
554,0,10000,TX,32905
555,10000,14999,TX,10771
556,15000,19999,TX,11228
557,20000,24999,TX,15778
558,25000,29999,TX,7482
559,30000,34999,TX,9914
560,35000,39999,TX,12678
561,40000,44999,TX,13268


In [189]:
rango1 = 30
rango2 = 34

In [190]:
income[((income['income_lower'] <= rango1) & (income['income_upper'] >= rango2)) & (income['Estados'] == 'TX') & ((income['income_lower'] != 0) & (income['income_upper'] != 10000))]

,income_lower,income_upper,Estados,cantidad
570,25,44,TX,1541890


In [191]:
pos=income[((income['income_lower'] <= rango1) & (income['income_upper'] >= rango2)) & (income['Estados'] == 'TX') & ((income['income_lower'] != 0) & (income['income_upper'] != 10000))].index
pos

Int64Index([570], dtype='int64')

In [192]:
r = income.iloc[pos[0]+1:pos[0]+17,]
r

,income_lower,income_upper,Estados,cantidad
571,0,10000,TX,44796
572,10000,14999,TX,21086
573,15000,19999,TX,25215
574,20000,24999,TX,32595
575,25000,29999,TX,32642
576,30000,34999,TX,41577
577,35000,39999,TX,41799
578,40000,44999,TX,43237
579,45000,49999,TX,40555
580,50000,59999,TX,107333


OBTENEMOS EL RANGO DE SALARON, EN BASE A UNA EDAD

In [193]:
num = r[r['cantidad'] == r["cantidad"].max()].index
pd.DataFrame(income.iloc[num[0]]).T

,income_lower,income_upper,Estados,cantidad
582,75000,99999,TX,220582


### -

In [194]:
filename = os.path.join(path,'Matrimonio.csv')
matrimonio = load_data(filename)

In [195]:
pasos = 1
matrimonio_data = matrimonio.iloc[2:16,[0]]
for cont in range(10):
    matrimonio_data.insert(matrimonio_data.shape[1], matrimonio.columns[pasos],matrimonio.iloc[2:16,[pasos]].apply(lambda x: x.str.replace(',', '')).astype(int), True)
    pasos += 1
    matrimonio_data.insert(matrimonio_data.shape[1], matrimonio.columns[pasos],matrimonio.iloc[2:16,[pasos]].apply(lambda x: x.str.replace('%', '')).astype(float)/100, True)
    pasos += 5

In [196]:
States = vector_states(matrimonio_data)
matrimonio_data.rename(columns = {'Label (Grouping)':'ageRange'}, inplace = True)
for i in list(matrimonio_data.columns.values):
    for j in States:
        if i.find(j) != -1:
            matrimonio_data.rename(columns={i:j},inplace=True)


In [197]:
dfMarriages = pd.DataFrame()
age_lower  = []
age_uper = []
bandera = True
for item in matrimonio_data.ageRange:
    numbers = re.findall(r'\d+', item)
    if (int(numbers[0]) == 15 and bandera):
        age_lower.append(-1)
        age_uper.append(int(numbers[0]))
        bandera = False
    else:
        age_lower.append(int(numbers[0]))
        if len(numbers) > 1:
            age_uper.append(int(numbers[1]))
        else:
            age_uper.append(-1)
dfMarriages['age_lower'] = age_lower
dfMarriages['age_uper'] = age_uper
dfMarriages = dfMarriages.iloc[0:int(dfMarriages.shape[0]/2),:]
dfMarriages

,age_lower,age_uper
0,-1,15
1,15,19
2,20,34
3,35,44
4,45,54
5,55,64
6,65,-1


In [198]:
paso = 1
for i in range(10):
    probabilidad = []
    for i in range(dfMarriages.shape[0]):
        pesos = []
        total = matrimonio_data.iloc[i,paso] + matrimonio_data.iloc[i+7,paso]
        pesos.append(matrimonio_data.iloc[i,paso]/total)
        pesos.append(matrimonio_data.iloc[i+7,paso]/total) 
        prob = (matrimonio_data.iloc[i,paso+1]*pesos[0]) + (matrimonio_data.iloc[i+7,paso+1]*pesos[1])
        if (prob >= 0.45):
            probabilidad.append("Yes")
        else:
            probabilidad.append("No")
    dfMarriages.insert(dfMarriages.shape[1], matrimonio_data.columns[paso],probabilidad, True)
    paso +=2
dfMarriages

,age_lower,age_uper,Arizona,Florida,Georgia,Illinois,Michigan,North Carolina,Ohio,Tennessee,Texas,Virginia
0,-1,15,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes
1,15,19,No,No,No,No,No,No,No,No,No,No
2,20,34,No,No,No,No,No,No,No,No,No,No
3,35,44,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes
4,45,54,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes
5,55,64,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes
6,65,-1,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes


In [199]:
dfMarriages = dfMarriages.drop(0)
dfMarriages = dfMarriages.melt(['age_lower', 'age_uper'], var_name='Estados', value_name='Probabilidad')
dfMarriages.head(10)


,age_lower,age_uper,Estados,Probabilidad
0,15,19,Arizona,No
1,20,34,Arizona,No
2,35,44,Arizona,Yes
3,45,54,Arizona,Yes
4,55,64,Arizona,Yes
5,65,-1,Arizona,Yes
6,15,19,Florida,No
7,20,34,Florida,No
8,35,44,Florida,Yes
9,45,54,Florida,Yes


In [200]:
iniciales = ['AZ','FL','GA','IL','MI','NC','OH','TN','TX','VA']
for i in range(len(States)):
    dfMarriages['Estados'] = dfMarriages['Estados'].str.replace(States[i], iniciales[i])

dfMarriages.head()

,age_lower,age_uper,Estados,Probabilidad
0,15,19,AZ,No
1,20,34,AZ,No
2,35,44,AZ,Yes
3,45,54,AZ,Yes
4,55,64,AZ,Yes


In [201]:
filename = os.path.join(path,'Households.csv')
hogares = load_data(filename)

In [202]:
hogares.head()

,Label (Grouping),Arizona!!Total!!Estimate,Arizona!!Married-couple family household!!Estimate,"Arizona!!Male householder, no spouse present, family household!!Estimate","Arizona!!Female householder, no spouse present, family household!!Estimate",Arizona!!Nonfamily household!!Estimate,Florida!!Total!!Estimate,Florida!!Married-couple family household!!Estimate,"Florida!!Male householder, no spouse present, family household!!Estimate","Florida!!Female householder, no spouse present, family household!!Estimate",...,Texas!!Total!!Estimate,Texas!!Married-couple family household!!Estimate,"Texas!!Male householder, no spouse present, family household!!Estimate","Texas!!Female householder, no spouse present, family household!!Estimate",Texas!!Nonfamily household!!Estimate,Virginia!!Total!!Estimate,Virginia!!Married-couple family household!!Estimate,"Virginia!!Male householder, no spouse present, family household!!Estimate","Virginia!!Female householder, no spouse present, family household!!Estimate",Virginia!!Nonfamily household!!Estimate
0,HOUSEHOLDS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Total households,"2,817,723","1,344,242","163,839","338,277","971,365","8,565,329","3,975,785","430,783","1,104,097",...,"10,796,247","5,341,490","571,198","1,460,497","3,423,062","3,331,461","1,629,590","153,061","382,293","1,166,517"
2,Average household size,2.53,3.16,3.15,3.31,1.29,2.49,3.10,3.31,3.39,...,2.68,3.36,3.20,3.39,1.24,2.52,3.19,3.22,3.26,1.26
3,FAMILIES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Total families,"1,846,358","1,344,242","163,839","338,277",(X),"5,510,665","3,975,785","430,783","1,104,097",...,"7,373,185","5,341,490","571,198","1,460,497",(X),"2,164,944","1,629,590","153,061","382,293",(X)


In [203]:
pasos = 1
hogares_data = pd.DataFrame(hogares.iloc[2:3,[0]])
for cont in range(10):
    hogares_data.insert(hogares_data.shape[1], hogares.columns[pasos],hogares.iloc[2:3,[pasos]].astype(float), True)
    pasos += 4
    hogares_data.insert(hogares_data.shape[1], hogares.columns[pasos],hogares.iloc[2:3,[pasos]].astype(float), True)
    pasos += 1
hogares_data

,Label (Grouping),Arizona!!Total!!Estimate,Arizona!!Nonfamily household!!Estimate,Florida!!Total!!Estimate,Florida!!Nonfamily household!!Estimate,Georgia!!Total!!Estimate,Georgia!!Nonfamily household!!Estimate,Illinois!!Total!!Estimate,Illinois!!Nonfamily household!!Estimate,Michigan!!Total!!Estimate,...,North Carolina!!Total!!Estimate,North Carolina!!Nonfamily household!!Estimate,Ohio!!Total!!Estimate,Ohio!!Nonfamily household!!Estimate,Tennessee!!Total!!Estimate,Tennessee!!Nonfamily household!!Estimate,Texas!!Total!!Estimate,Texas!!Nonfamily household!!Estimate,Virginia!!Total!!Estimate,Virginia!!Nonfamily household!!Estimate
2,Average household size,2.53,1.29,2.49,1.27,2.64,1.25,2.48,1.22,2.43,...,2.46,1.23,2.38,1.23,2.46,1.23,2.68,1.24,2.52,1.26


In [204]:
States = vector_states(hogares_data)
hogares_data.rename(columns = {'Label (Grouping)':'ageRange'}, inplace = True)
for i in list(hogares_data.columns.values):
    for j in States:
        if i.find(j) != -1:
            hogares_data.rename(columns={i:j},inplace=True)

In [205]:
hogares_data

,ageRange,Arizona,Arizona,Florida,Florida,Georgia,Georgia,Illinois,Illinois,Michigan,...,North Carolina,North Carolina,Ohio,Ohio,Tennessee,Tennessee,Texas,Texas,Virginia,Virginia
2,Average household size,2.53,1.29,2.49,1.27,2.64,1.25,2.48,1.22,2.43,...,2.46,1.23,2.38,1.23,2.46,1.23,2.68,1.24,2.52,1.26


In [206]:
paso = 1
dfHouseholds = pd.DataFrame()
lower_limit_casados = []
upper_limit_casados = []
lower_limit_solteros = []
upper_limit_solteros = []
for i in range(10):
    if (hogares_data.iloc[0,paso] - hogares_data.iloc[0,paso].astype(int) > 0.5):
        lower_limit_casados.append(hogares_data.iloc[0,paso].astype(int))
        upper_limit_casados.append(hogares_data.iloc[0,paso].astype(int)+1)
    else:
        lower_limit_casados.append(hogares_data.iloc[0,paso].astype(int))
        upper_limit_casados.append(hogares_data.iloc[0,paso].astype(int))
    paso += 1
    if (hogares_data.iloc[0,paso] - hogares_data.iloc[0,paso].astype(int) > 0.5):
        lower_limit_solteros.append(hogares_data.iloc[0,paso].astype(int))
        upper_limit_solteros.append(hogares_data.iloc[0,paso].astype(int)+1)
    else:
        lower_limit_solteros.append(hogares_data.iloc[0,paso].astype(int))
        upper_limit_solteros.append(hogares_data.iloc[0,paso].astype(int))
    paso += 1

dfHouseholds['State'] = States
dfHouseholds['limiteInfCasados'] = lower_limit_casados
dfHouseholds['limiteSupCasados'] = upper_limit_casados
dfHouseholds['limiteInfSolteros'] = lower_limit_solteros
dfHouseholds['limiteSupSolteros'] = upper_limit_solteros
dfHouseholds


,State,limiteInfCasados,limiteSupCasados,limiteInfSolteros,limiteSupSolteros
0,Arizona,2,3,1,1
1,Florida,2,2,1,1
2,Georgia,2,3,1,1
3,Illinois,2,2,1,1
4,Michigan,2,2,1,1
5,North Carolina,2,2,1,1
6,Ohio,2,2,1,1
7,Tennessee,2,2,1,1
8,Texas,2,3,1,1
9,Virginia,2,3,1,1


In [207]:
iniciales = ['AZ','FL','GA','IL','MI','NC','OH','TN','TX','VA']
for i in range(len(States)):
    dfHouseholds['State'] = dfHouseholds['State'].str.replace(States[i], iniciales[i])

dfHouseholds

,State,limiteInfCasados,limiteSupCasados,limiteInfSolteros,limiteSupSolteros
0,AZ,2,3,1,1
1,FL,2,2,1,1
2,GA,2,3,1,1
3,IL,2,2,1,1
4,MI,2,2,1,1
5,NC,2,2,1,1
6,OH,2,2,1,1
7,TN,2,2,1,1
8,TX,2,3,1,1
9,VA,2,3,1,1


Primero se obtiene el rango de edad con más población del estado (Top 3) <br>
Luego con ellos:<br>
- Income más probable con ese rango de edad
- Su estatus matrimonial con la % de ese rango
- Su householdsize en base al estatus matrimonial 

Tablas:
- income
- dfMarriages
- age_data
- dfHouseholds

In [208]:
income = income.rename(columns={'Estados': 'StateCode'})
dfMarriages = dfMarriages.rename(columns={'Estados': 'StateCode'})
age_data = age_data.rename(columns={'Estados': 'StateCode'})
dfHouseholds = dfHouseholds.rename(columns={'State': 'StateCode'})

Creación de tabla perfiles

In [209]:
perfiles = pd.DataFrame({'StateCode':[], 'age_lower':[], 'age_upper':[], 'income_lower':[], 'income_upper':[], 'status_matrimonial':[], 'limitinf_house':[], 'limisup_house':[]})
perfiles

,StateCode,age_lower,age_upper,income_lower,income_upper,status_matrimonial,limitinf_house,limisup_house


In [210]:
# select * from age_data where statecode == 'AZ' order by porcentaje desc limit 3
for i in ['TX','FL','IL','OH','NC','GA','MI','VA','AZ','TN']:
    perfiles = pd.concat([perfiles, age_data[(age_data['StateCode'] == i) & (age_data["age_lower"] >= 25)].sort_values('porcentaje', ascending=False).head(3)[['StateCode', 'age_lower', 'age_upper']].head(3)])

perfiles = perfiles.reset_index()
perfiles = perfiles.drop(columns=["index"])
perfiles.head(3)

,StateCode,age_lower,age_upper,income_lower,income_upper,status_matrimonial,limitinf_house,limisup_house
0,TX,30.0,34.0,NaN,NaN,NaN,NaN,NaN
1,TX,35.0,39.0,NaN,NaN,NaN,NaN,NaN
2,TX,25.0,29.0,NaN,NaN,NaN,NaN,NaN


In [211]:
res = pd.DataFrame()
for index, row in perfiles.iterrows():
    pos=income[((income['income_lower'] <= row["age_lower"]) & (income['income_upper'] >= row["age_upper"])) & (income['StateCode'] == row["StateCode"]) & ((income['income_lower'] != 0) & (income['income_upper'] != 10000))].index
    r = income.iloc[pos[0]+1:pos[0]+17,]
    num = r[r['cantidad'] == r["cantidad"].max()].index
    res = pd.concat([res,pd.DataFrame(income.iloc[num[0]]).T[["income_lower", "income_upper"]] ]) 
res = res.reset_index()
res = res.drop(columns=["index"])
res

perfiles["income_lower"] =  res.iloc[:,0]
perfiles["income_upper"] =  res.iloc[:,1]
perfiles.head(3)
    

,StateCode,age_lower,age_upper,income_lower,income_upper,status_matrimonial,limitinf_house,limisup_house
0,TX,30.0,34.0,75000,99999,NaN,NaN,NaN
1,TX,35.0,39.0,75000,99999,NaN,NaN,NaN
2,TX,25.0,29.0,75000,99999,NaN,NaN,NaN


In [212]:
res = pd.DataFrame()
for index, row in perfiles.iterrows():
    pos=dfMarriages[((dfMarriages['age_lower'] <= row["age_lower"]) & (dfMarriages['age_uper'] >= row["age_upper"])) & (dfMarriages['StateCode'] == row["StateCode"])].index
    res = pd.concat([res, pd.DataFrame(dfMarriages.iloc[pos])["Probabilidad"]]) 
res = res.reset_index()
res = res.drop(columns=["index"])
res

perfiles["status_matrimonial"] =  res.iloc[:,0]
perfiles.head(3)

,StateCode,age_lower,age_upper,income_lower,income_upper,status_matrimonial,limitinf_house,limisup_house
0,TX,30.0,34.0,75000,99999,No,NaN,NaN
1,TX,35.0,39.0,75000,99999,Yes,NaN,NaN
2,TX,25.0,29.0,75000,99999,No,NaN,NaN


In [213]:
inferior = []
superior = []
for index, row in perfiles.iterrows():
    pos=dfHouseholds[(dfHouseholds['StateCode'] == row["StateCode"])].index
    if row["status_matrimonial"] == "Yes":
        inferior.append(dfHouseholds.iat[8,1])
        superior.append(dfHouseholds.iat[8,2])    
    else:
        inferior.append(dfHouseholds.iat[8,3])
        superior.append(dfHouseholds.iat[8,4])  

perfiles["limitinf_house"] = inferior
perfiles["limisup_house"] = superior

Perfiles

In [214]:
perfiles["income_lower"] = perfiles.iloc[:,3].apply(lambda x: (x/12)*0.10 if x > 0 else -1)
perfiles["income_lower"] = perfiles["income_lower"].round(2)
perfiles["income_upper"] = perfiles.iloc[:,4].apply(lambda x: (x/12)*0.10 if x > 0 else -1)
perfiles["income_upper"] = perfiles["income_upper"].round(2)
perfiles["age_lower"] = perfiles["age_lower"].astype(int)
perfiles["age_upper"] = perfiles["age_upper"].astype(int)
perfiles

,StateCode,age_lower,age_upper,income_lower,income_upper,status_matrimonial,limitinf_house,limisup_house
0,TX,30,34,625.00,833.32,No,1,1
1,TX,35,39,625.00,833.32,Yes,2,3
2,TX,25,29,625.00,833.32,No,1,1
3,FL,60,64,1666.67,-1.00,Yes,2,3
4,FL,55,59,1666.67,-1.00,Yes,2,3
5,FL,30,34,625.00,833.32,No,1,1
6,IL,30,34,625.00,833.32,No,1,1
7,IL,35,39,625.00,833.32,Yes,2,3
8,IL,25,29,625.00,833.32,No,1,1
9,OH,60,64,625.00,833.32,Yes,2,3


In [215]:

perfiles = perfiles.assign(idperfil = lambda df: df.index+1)
perfiles = perfiles[['idperfil', 'StateCode', 'age_lower', 'age_upper', 'income_lower', 'income_upper',
       'status_matrimonial', 'limitinf_house', 'limisup_house']]



# Insert de tablas en la base de datos

In [402]:
"""#Proceso de determinar posibles planes
rango 20-25 años
rango 0-10,000 ingreso anual
rango 0-833 ingreso mensual
rango 0-83
Obtener todos los StandardComponentid (unicos) que: (age>=20 y age<=25) y (IndivialRate>=0 y IndivialRate<=83)
En la tabla planes, traer todos los planes de esos StandardComponentID 

# criterio para escoger el plan
Se basa en lo que tiene la tabla Planes y Planes"""

'#Proceso de determinar posibles planes\nrango 20-25 años\nrango 0-10,000 ingreso anual\nrango 0-833 ingreso mensual\nrango 0-83\nObtener todos los StandardComponentid (unicos) que: (age>=20 y age<=25) y (IndivialRate>=0 y IndivialRate<=83)\nEn la tabla planes, traer todos los planes de esos StandardComponentID \n\n# criterio para escoger el plan\nSe basa en lo que tiene la tabla Planes y Planes'

In [404]:
df_ejemplo = pd.DataFrame({'StandardComponentId_PlanId': ['87226TX0050005'], 'AVG(IndividualRate)': [300], 'idperfil': [1]})
df_ejemplo

,StandardComponentId_PlanId,AVG(IndividualRate),idperfil
0,87226TX0050005,300,1


In [405]:
# select airport_ident, type, description, frequency_mhz from airport_freq join airports on airport_freq.airport_ref = airports.id where airports.ident = 'KLAX'
#planes
#airport_freq.merge(airports[airports.ident == 'KLAX'][['id']], left_on='airport_ref', right_on='id', how='inner')[['airport_ident', 'type', 'description', 'frequency_mhz']]
Planes.merge(df_ejemplo, left_on='StandardComponentId', right_on='StandardComponentId_PlanId', how='inner')[['StateCode','idplan', 'AVG(IndividualRate)', 'idperfil']]

,StateCode,idplan,AVG(IndividualRate),idperfil
0,TX,13353,300,1
1,TX,13354,300,1
2,TX,13355,300,1
3,TX,13356,300,1
4,TX,13357,300,1
5,TX,13358,300,1
6,TX,13359,300,1


In [408]:
d = pd.DataFrame({'StandardComponentId_PlanId': ['87226TX0050005', '87226TX0050006'], 'AVG(IndividualRate)': [300,500]})
d

,StandardComponentId_PlanId,AVG(IndividualRate)
0,87226TX0050005,300
1,87226TX0050006,500


In [216]:
import psycopg2
import psycopg2.extras as extras
import pandas as pd

In [247]:
conn = psycopg2.connect(
  database="proyectoBI",
  user='postgres', 
  password='admin', 
  host='localhost', 
  port= '5432'
)
plan_perfil = pd.DataFrame()
cursor = conn.cursor()
for index, row in perfiles.iterrows():
    if (row["income_upper"] != -1):
        postgreSQL_select_Query = '''select "stdId", AVG("IndividualRate") 
                                    from public."rates" 
                                    Where (("Age" >= %s) and ("Age" <= %s)) and (("IndividualRate" >= %s) and ("IndividualRate" <= %s)) and ("StateCode" = %s) 
                                    Group by "stdId"'''
        cursor.execute(postgreSQL_select_Query, (row["age_lower"],row["age_upper"],row["income_lower"], row["income_upper"],row["StateCode"],))
        results = cursor.fetchall()
        df = pd.DataFrame(results, columns=[desc[0] for desc in cursor.description])
        # STCOMP|AVG
        # Agregar el ID del cliente
        df['idperfil'] = row['idperfil']
    else:
        postgreSQL_select_Query = '''select "stdId", AVG("IndividualRate") 
                                  from public."rates" 
                                  Where (("Age" >= %s) and ("Age" <= %s)) and (("IndividualRate" <= %s)) and ("StateCode" = %s) 
                                  Group by "stdId"'''
        
        cursor.execute(postgreSQL_select_Query, (row["age_lower"],row["age_upper"],row["income_lower"],row["StateCode"],))
        results = cursor.fetchall()
        df = pd.DataFrame(results, columns=[desc[0] for desc in cursor.description])
        # STCOMP|AVG
        # Agregar el ID del cliente
        df['idperfil'] = row['idperfil']
    
    plan_perfil = pd.concat([plan_perfil, Planes.merge(df, left_on='StandardComponentId', right_on='stdId', how='inner')[['StateCode','idplan', 'avg', 'idperfil']]])
    
cursor.close()
conn.close()
plan_perfil
    

,StateCode,idplan,avg,idperfil
0,TX,9165,642.7977777777777778,1
1,TX,9166,642.7977777777777778,1
2,TX,9167,642.7977777777777778,1
3,TX,9168,642.7977777777777778,1
4,TX,9169,642.7977777777777778,1
...,...,...,...,...
283,TN,8836,767.1960000000000000,30
284,TN,8919,739.6660000000000000,30
285,TN,8920,739.6660000000000000,30
286,TN,8921,739.6660000000000000,30


In [248]:
plan_perfil = plan_perfil.rename(columns={'avg':'rate'})

In [218]:
import psycopg2
import psycopg2.extras as extras
import pandas as pd


def execute_values(conn, df, table):

	tuples = [tuple(x) for x in df.to_numpy()]

	cols = ','.join(list(df.columns))
	finish = True
	coma = 0
	cont = 0
	cols_final = '"'
	cols += ","
	while(finish):
		coma = cols.find(",", cont)
		if coma == -1:
			finish = False
			cols_final = cols_final[:-2]
		else:
			var = cols[cont:coma]
			cont = coma + 1
			cols_final += var
			cols_final += '","'


	print(cols_final)
	# SQL query to execute
	query = "INSERT INTO %s(%s) VALUES %%s" % (table, cols_final)
	cursor = conn.cursor()
	try:
		extras.execute_values(cursor, query, tuples)
		conn.commit()
	except (Exception, psycopg2.DatabaseError) as error:
		print("Error: %s" % error)
		conn.rollback()
		cursor.close()
		return 1
	print("execute_values() done")
	cursor.close()

Insert tabla Profiles

In [257]:
# establishing connection
conn = psycopg2.connect(
  database="proyectoBI",
  user='postgres', 
  password='admin', 
  host='localhost', 
  port= '5432'
)
sql = '''DELETE FROM public."profiles";'''
  
#creating a cursor
cursor = conn.cursor()
cursor.execute(sql)
conn.commit()
data = perfiles
# using the function defined
execute_values(conn, data, 'public."profiles"')
conn.close()

"idperfil","StateCode","age_lower","age_upper","income_lower","income_upper","status_matrimonial","limitinf_house","limisup_house"
execute_values() done


Insert tabla Bussiness Rules

In [222]:
# establishing connection
conn = psycopg2.connect(
  database="proyectoBI",
  user='postgres', 
  password='admin', 
  host='localhost', 
  port= '5432'
)
sql = '''DELETE FROM public."businessRules";'''
  
#creating a cursor
cursor = conn.cursor()
cursor.execute(sql)
conn.commit()
data = Business_Rules
# using the function defined
execute_values(conn, data, 'public."businessRules"')
conn.close()

"StateCode","IssuerId","MedicalDentalBoth","StandardComponentId","MaximumUnderageDependent","MarketCoverage","DentalOnlyPlan"
execute_values() done


Insert tabla Benefits_CostSharing

In [226]:
# establishing connection
conn = psycopg2.connect(
  database="proyectoBI",
  user='postgres', 
  password='admin', 
  host='localhost', 
  port= '5432'
)
sql = '''DELETE FROM public."benefitsCostSharing";'''
  
#creating a cursor
cursor = conn.cursor()
cursor.execute(sql)
conn.commit()
data = Benefits_CostSharing
# using the function defined
execute_values(conn, data, 'public."benefitsCostSharing"')
conn.close()

"idBeneficio","StateCode","IssuerId","StandardComponentId","PlanId","BenefitName","CopayInnTier1","CopayInnTier2","CopayOutofNet","CoinsInnTier1","CoinsInnTier2","CoinsOutofNet","IsEHB","IsCovered","QuantLimitOnSvc","LimitQty","LimitUnit","Exclusions"
execute_values() done


Insert tabla Rate

In [227]:
# establishing connection
conn = psycopg2.connect(
  database="proyectoBI",
  user='postgres', 
  password='admin', 
  host='localhost', 
  port= '5432'
)
sql = '''DELETE FROM public."rates";'''
  
#creating a cursor
cursor = conn.cursor()
cursor.execute(sql)
conn.commit()
data = Rate
# using the function defined
execute_values(conn, data, 'public."rates"')
conn.close()

"idRate","StateCode","IssuerId","StandardComponentId_PlanId","Age","IndividualRate","Couple","PrimarySubscriberAndOneDependent","PrimarySubscriberAndTwoDependents","PrimarySubscriberAndThreeOrMoreDependents","CoupleAndOneDependent","CoupleAndTwoDependents","CoupleAndThreeOrMoreDependents"
execute_values() done


Insert tabla Plan Attributes

In [228]:
# establishing connection
conn = psycopg2.connect(
  database="proyectoBI",
  user='postgres', 
  password='admin', 
  host='localhost', 
  port= '5432'
)
sql = '''DELETE FROM public."planAttributes";'''
  
#creating a cursor
cursor = conn.cursor()
cursor.execute(sql)
conn.commit()
data = Plan_Attributes
# using the function defined
execute_values(conn, data, 'public."planAttributes"')
conn.close()

"StateCode","IssuerId","IssuerMarketPlaceMarketingName","MarketCoverage","StandardComponentId","PlanMarketingName","IsNewPlan","MetalLevel","ChildOnlyOffering","DiseaseManagementProgramsOffered","EHBPercentTotalPremium","PlanId","IssuerActuarialValue","IsHSAEligible"
execute_values() done


In [263]:
# establishing connection
conn = psycopg2.connect(
  database="proyectoBI",
  user='postgres', 
  password='admin', 
  host='localhost', 
  port= '5432'
)
sql = '''DELETE FROM public."plans";'''
  
#creating a cursor
cursor = conn.cursor()
cursor.execute(sql)
conn.commit()
data = Planes
# using the function defined
execute_values(conn, data, 'public."plans"')
conn.close()

"idplan","StateCode","StandardComponentId","PlanId","MetalLevel","PlanMarketingName","IsNewPlan","MaximumUnderageDependent","MarketCoverage","ChildOnlyOffering","DiseaseManagementProgramsOffered","EHBPercentTotalPremium","IsHSAEligible"
execute_values() done


In [252]:
# establishing connection
conn = psycopg2.connect(
  database="proyectoBI",
  user='postgres', 
  password='admin', 
  host='localhost', 
  port= '5432'
)
sql = '''DELETE FROM public."ages";'''
  
#creating a cursor
cursor = conn.cursor()
cursor.execute(sql)
conn.commit()
data = age_data
# using the function defined
execute_values(conn, data, 'public."ages"')
conn.close()

"StateCode","age_lower","age_upper","porcentaje"
execute_values() done


In [254]:
# establishing connection
conn = psycopg2.connect(
  database="proyectoBI",
  user='postgres', 
  password='admin', 
  host='localhost', 
  port= '5432'
)
sql = '''DELETE FROM public."households_sizes";'''
  
#creating a cursor
cursor = conn.cursor()
cursor.execute(sql)
conn.commit()
data = dfHouseholds
# using the function defined
execute_values(conn, data, 'public."households_sizes"')
conn.close()

"StateCode","limiteInfCasados","limiteSupCasados","limiteInfSolteros","limiteSupSolteros"
execute_values() done


In [255]:
# establishing connection
conn = psycopg2.connect(
  database="proyectoBI",
  user='postgres', 
  password='admin', 
  host='localhost', 
  port= '5432'
)
sql = '''DELETE FROM public."marriages";'''
  
#creating a cursor
cursor = conn.cursor()
cursor.execute(sql)
conn.commit()
data = dfMarriages
# using the function defined
execute_values(conn, data, 'public."marriages"')
conn.close()

"age_lower","age_uper","StateCode","Probabilidad"
execute_values() done


In [256]:
# establishing connection
conn = psycopg2.connect(
  database="proyectoBI",
  user='postgres', 
  password='admin', 
  host='localhost', 
  port= '5432'
)
sql = '''DELETE FROM public."incomes";'''
  
#creating a cursor
cursor = conn.cursor()
cursor.execute(sql)
conn.commit()
data = income
# using the function defined
execute_values(conn, data, 'public."incomes"')
conn.close()

"income_lower","income_upper","StateCode","cantidad"
execute_values() done


In [264]:
# establishing connection
conn = psycopg2.connect(
  database="proyectoBI",
  user='postgres', 
  password='admin', 
  host='localhost', 
  port= '5432'
)
sql = '''DELETE FROM public."plans_profiles";'''
  
#creating a cursor
cursor = conn.cursor()
cursor.execute(sql)
conn.commit()
data = plan_perfil
# using the function defined
execute_values(conn, data, 'public."plans_profiles"')
conn.close()

"StateCode","idplan","rate","idperfil"
execute_values() done


In [268]:
# establishing connection
conn = psycopg2.connect(
  database="proyectoBI",
  user='postgres', 
  password='admin', 
  host='localhost', 
  port= '5432'
)
sql = '''DELETE FROM public."plans_benefits";'''
  
#creating a cursor
cursor = conn.cursor()
cursor.execute(sql)
conn.commit()
data = Planes_beneficios
# using the function defined
execute_values(conn, data, 'public."plans_benefits"')
conn.close()

"idBeneficio","idplan","BenefitName","IsEHB","IsCovered"
execute_values() done


In [267]:
Planes_beneficios

,idBeneficio,idplan,BenefitName,IsEHB,IsCovered
0,241,1,Primary Care Visit to Treat an Injury or Illness,Yes,Covered
1,242,1,Specialist Visit,Yes,Covered
2,243,1,"Other Practitioner Office Visit (Nurse, Physic...",Yes,Covered
3,244,1,"Outpatient Facility Fee (e.g., Ambulatory Surg...",Yes,Covered
4,245,1,Outpatient Surgery Physician/Surgical Services,Yes,Covered
...,...,...,...,...,...
1055542,1358439,19188,Prosthetic Devices,Yes,Covered
1055543,1358440,19188,Infusion Therapy,Yes,Covered
1055544,1358441,19188,Treatment for Temporomandibular Joint Disorders,Yes,Covered
1055545,1358442,19188,Nutritional Counseling,Yes,Covered
